# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

In [7]:
file = pd.read_csv("../output_files/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Otradnoye,100,RU,1650816859,78,59.7775,30.8181,48.87,9.62
1,1,Castro,39,BR,1650816887,61,-24.7911,-50.0119,75.45,2.26
2,2,Tommot,100,RU,1650816888,96,58.9564,126.2925,23.79,1.07
3,3,Mataura,100,NZ,1650816888,89,-46.1927,168.8643,52.48,6.11
4,4,Vardø,100,NO,1650816888,89,70.3705,31.1107,37.65,11.32


## Part 1: Humidity Heat Map 

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [9]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [12]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
#Changing the temp criteria from 70-80F to 65-85F for better variety

temp_file=file.loc[(file['Max Temp']>=65)&(file['Max Temp']<=85)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']<=0)]
ideal_weather.head(23)

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
34,34,Lompoc,0,US,1650816828,69,34.6391,-120.4579,70.93,4.00
82,82,Shetpe,0,KZ,1650816914,30,44.1667,52.1167,66.49,8.19
96,96,Guerrero Negro,0,MX,1650816919,60,27.9769,-114.0611,65.57,4.34
116,116,Inhambane,0,MZ,1650816926,79,-23.8650,35.3833,74.68,6.26
133,133,Cabo San Lucas,0,MX,1650816934,79,22.8909,-109.9124,78.62,3.44
151,151,Laguna,0,US,1650816942,62,38.4210,-121.4238,65.32,5.75
208,208,Ixtapa,0,MX,1650816962,69,20.7000,-105.2000,76.23,0.00
243,243,Tura,0,IN,1650816975,56,25.5198,90.2201,74.80,5.28
265,265,Acapulco de Juárez,0,MX,1650816910,65,16.8634,-99.8901,84.02,8.05
311,311,Christiansburg,0,US,1650816732,41,37.1298,-80.4089,81.46,6.91


In [36]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# create search dictionary to update each iteration
search = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": gkey
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    search["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=search)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df



{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.6574784,
                    "lng": -120.4584883
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.65866132989272,
                        "lng": -120.4574347
                    },
                    "southwest": {
                        "lat": 34.65596167010727,
                        "lng": -120.4603883
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hilton Garden Inn Lompoc",
            "photos": [
                {
                    "height": 4000,
 

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.001048,
                    "lng": -114.0138683
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.00238187989272,
                        "lng": -114.0121708701073
                    },
                    "southwest": {
                        "lat": 27.99968222010728,
                        "lng": -114.0148705298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.8622222,
                    "lng": 35.3783333
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.86095732010727,
                        "lng": 35.37973827989272
                    },
                    "southwest": {
                        "lat": -23.86365697989272,
                        "lng": 35.37703862010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Casa Do Capit\u00e3o",
            "photos": [
                {
                    "hei

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.4266746,
                    "lng": -121.4821789
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.42795052989273,
                        "lng": -121.4805719701073
                    },
                    "southwest": {
                        "lat": 38.42525087010728,
                        "lng": -121.4832716298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5",
            "photos": [
               

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.5144682,
                    "lng": 90.1922063
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.51585602989272,
                        "lng": 90.19346087989273
                    },
                    "southwest": {
                        "lat": 25.51315637010728,
                        "lng": 90.19076122010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Polo Orchid",
            "opening_hours": {
                "open_now": true
            },

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.7121558,
                    "lng": -111.4993896
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.71356267989272,
                        "lng": -111.4979826201073
                    },
                    "southwest": {
                        "lat": 32.71086302010728,
                        "lng": -111.5006822798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png",
            "icon_background_color": "#4DB546",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/camping_pinlet",
            "name": "Picacho / Tucson NW KOA Journey",
            "opening_hours": {
                "open_now":

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.752417,
                    "lng": 116.8487121
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.75109842010728,
                        "lng": 116.8503656798927
                    },
                    "southwest": {
                        "lat": -20.75379807989272,
                        "lng": 116.8476660201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Best Western Plus The Ranges Karratha",
            "opening_hours": {
                "open_no

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.819656,
                    "lng": 115.8329
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.82126472989272,
                        "lng": 115.8342612298927
                    },
                    "southwest": {
                        "lat": 22.81856507010728,
                        "lng": 115.8315615701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
       

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.975354,
                    "lng": -46.5356684
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.97407442010728,
                        "lng": -46.53427692010727
                    },
                    "southwest": {
                        "lat": -22.97677407989272,
                        "lng": -46.53697657989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "KA Business Hotel",
            "opening_hours": {
                "open_now": true
         

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,34,Lompoc,0,US,1650816828,69,34.6391,-120.4579,70.93,4.00,Hilton Garden Inn Lompoc
1,82,Shetpe,0,KZ,1650816914,30,44.1667,52.1167,66.49,8.19,
2,96,Guerrero Negro,0,MX,1650816919,60,27.9769,-114.0611,65.57,4.34,The Halfway Inn
3,116,Inhambane,0,MZ,1650816926,79,-23.8650,35.3833,74.68,6.26,Hotel Casa Do Capitão
4,133,Cabo San Lucas,0,MX,1650816934,79,22.8909,-109.9124,78.62,3.44,Medano Hotel and Suites
5,151,Laguna,0,US,1650816942,62,38.4210,-121.4238,65.32,5.75,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
6,208,Ixtapa,0,MX,1650816962,69,20.7000,-105.2000,76.23,0.00,Flamingo Vallarta Hotel & Marina
7,243,Tura,0,IN,1650816975,56,25.5198,90.2201,74.80,5.28,Hotel Polo Orchid
8,265,Acapulco de Juárez,0,MX,1650816910,65,16.8634,-99.8901,84.02,8.05,Hotel Emporio Acapulco
9,311,Christiansburg,0,US,1650816732,41,37.1298,-80.4089,81.46,6.91,Hampton Inn Christiansburg/Blacksburg


In [34]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))